<a href="https://colab.research.google.com/github/emiliagyr/EstadisticaExperimental/blob/main/Colab/2vias_sol2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pingouin
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 22.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import seaborn as sns
import pingouin as pg

Variable independiente 1

H0: u1 = U2 = U3 = U4 (las mediciones son iguales)
H1: Al menos 2 son distintos

Variable independiente 2

H0: u1 = U2 = U3 = U4 (las mediciones son iguales)
H1: Al menos 2 son distintos

Interacción

H0: No hay interacción
H1: Si hay interacción

#1. Leer los datos del documento .sav (SPSS) o .csv (Excel)

In [3]:
df = pd.read_spss("VIPP-SD.sav")
df

,Sujeto,NSE,Pre,Post,Post06m,Post12m
0,1.0,Medio,16.0,18.0,18.0,17.0
1,2.0,Medio,16.0,18.0,18.0,17.0
2,3.0,Bajo,12.0,14.0,14.0,13.0
3,4.0,Medio,15.0,18.0,18.0,17.0
4,5.0,Bajo,12.0,15.0,15.0,15.0
...,...,...,...,...,...,...
115,116.0,Bajo,12.0,15.0,15.0,14.0
116,117.0,Bajo,12.0,16.0,15.0,14.0
117,118.0,Bajo,13.0,16.0,15.0,14.0
118,119.0,Alto,13.0,16.0,16.0,15.0


##2. Es necesario crear una columna de las mediciones x intervencion para tener todos los puntajes de sensibilidad juntos

In [4]:
melt_df = pd.melt(
    df, value_name="Sensibilidad", var_name="Medxinter", id_vars=["NSE", "Sujeto"]
)
melt_df

,NSE,Sujeto,Medxinter,Sensibilidad
0,Medio,1.0,Pre,16.0
1,Medio,2.0,Pre,16.0
2,Bajo,3.0,Pre,12.0
3,Medio,4.0,Pre,15.0
4,Bajo,5.0,Pre,12.0
...,...,...,...,...
475,Bajo,116.0,Post12m,14.0
476,Bajo,117.0,Post12m,14.0
477,Bajo,118.0,Post12m,14.0
478,Alto,119.0,Post12m,15.0


##3. Pruebas de homogeneidad

Prueba de esfericidad de Mauchly
(Homogeneidad de varianzas para intrasujeto)

In [7]:
pg.sphericity(data=melt_df, dv='Sensibilidad', subject='Sujeto', within='Medxinter')

SpherResults(spher=False, W=0.7244465475355981, chi2=37.94744029049675, dof=5, pval=3.8720094008111637e-07)

Prueba de igualdad de Levene (Homogeneidad de varianzas para intersujeto)

In [9]:
pg.homoscedasticity(data=melt_df, dv='Sensibilidad', group='NSE')

,W,pval,equal_var
levene,68.658053,6.239715e-27,False


Por lo tanto rechazamos la homogeneidad de las varianzas, son varianzas heterogéneas

##4. Anova 2 vías

In [6]:
pg.anova(dv="Sensibilidad", between=["Medxinter", "NSE"], data=melt_df)

,Source,SS,DF,MS,F,p-unc,np2
0,Medxinter,524.983333,3.0,174.994444,47.619732,7.121085e-27,0.233866
1,NSE,112.643388,2.0,56.321694,15.326338,3.570768e-07,0.061471
2,Medxinter * NSE,44.519257,6.0,7.419876,2.019107,6.171433e-02,0.025233
3,Residual,1719.820689,468.0,3.674831,NaN,NaN,NaN
